In [1]:
import pandas as pd
from sklearn import svm
from sklearn import cross_validation
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.naive_bayes import BernoulliNB
import numpy as np
import matplotlib.pyplot as plt

In [2]:
loan = pd.read_csv('train.csv')
loan_credit_history = pd.read_csv('train.csv')
loan_test = pd.read_csv('test.csv')

In [3]:
loan.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [4]:
loan.count()

Loan_ID              614
Gender               601
Married              611
Dependents           599
Education            614
Self_Employed        582
ApplicantIncome      614
CoapplicantIncome    614
LoanAmount           592
Loan_Amount_Term     600
Credit_History       564
Property_Area        614
Loan_Status          614
dtype: int64

In [5]:
loan_test.count()

Loan_ID              367
Gender               356
Married              367
Dependents           357
Education            367
Self_Employed        344
ApplicantIncome      367
CoapplicantIncome    367
LoanAmount           362
Loan_Amount_Term     361
Credit_History       338
Property_Area        367
dtype: int64

In [6]:
loan.loc[loan['Property_Area'] == 'Rural', 'Property_Area'] = 0
loan.loc[loan['Property_Area'] == 'Semiurban', 'Property_Area'] = 1
loan.loc[loan['Property_Area'] == 'Urban', 'Property_Area'] = 2

loan.loc[loan['Education'] == 'Graduate', 'Education'] = 1
loan.loc[loan['Education'] == 'Not Graduate', 'Education'] = 0
loan_credit_history.loc[loan_credit_history['Education'] == 'Graduate', 'Education'] = 1
loan_credit_history.loc[loan_credit_history['Education'] == 'Not Graduate', 'Education'] = 0
loan.loc[loan['Married'] == 'Yes', 'Married'] = 1
loan.loc[loan['Married'] == 'No', 'Married'] = 0
loan.loc[loan['Gender'] == 'Male', 'Gender'] = 1
loan.loc[loan['Gender'] == 'Female', 'Gender'] = 0
loan.loc[loan['Self_Employed'] == 'Yes', 'Self_Employed'] = 1
loan.loc[loan['Self_Employed'] == 'No', 'Self_Employed'] = 0
loan.loc[loan['Dependents'] == '0', 'Dependents'] = 0
loan.loc[loan['Dependents'] == '1', 'Dependents'] = 1
loan.loc[loan['Dependents'] == '2', 'Dependents'] = 2
loan.loc[loan['Dependents'] == '3+', 'Dependents'] = 3

loan_test.loc[loan_test['Property_Area'] == 'Rural', 'Property_Area'] = 0
loan_test.loc[loan_test['Property_Area'] == 'Semiurban', 'Property_Area'] = 1
loan_test.loc[loan_test['Property_Area'] == 'Urban', 'Property_Area'] = 2
loan_test.loc[loan_test['Education'] == 'Graduate', 'Education'] = 1
loan_test.loc[loan_test['Education'] == 'Not Graduate', 'Education'] = 0
loan_test.loc[loan_test['Married'] == 'Yes', 'Married'] = 1
loan_test.loc[loan_test['Married'] == 'No', 'Married'] = 0
loan_test.loc[loan_test['Gender'] == 'Male', 'Gender'] = 1
loan_test.loc[loan_test['Gender'] == 'Female', 'Gender'] = 0
loan_test.loc[loan_test['Self_Employed'] == 'Yes', 'Self_Employed'] = 1
loan_test.loc[loan_test['Self_Employed'] == 'No', 'Self_Employed'] = 0
loan_test.loc[loan_test['Dependents'] == '0', 'Dependents'] = 0
loan_test.loc[loan_test['Dependents'] == '1', 'Dependents'] = 1
loan_test.loc[loan_test['Dependents'] == '2', 'Dependents'] = 2
loan_test.loc[loan_test['Dependents'] == '3+', 'Dependents'] = 3

loan['Credit_History'] = loan['Credit_History'].fillna(loan['Credit_History'].mean())
#loan['Credit_History'] = loan['Credit_History'].fillna(loan['Credit_History'].quantile())
loan['ApplicantIncome'] = loan['ApplicantIncome'].fillna(loan['ApplicantIncome'].quantile())
loan['LoanAmount'] = loan['LoanAmount'].fillna(loan['LoanAmount'].quantile())
loan['Loan_Amount_Term'] = loan['Loan_Amount_Term'].fillna(loan['Loan_Amount_Term'].quantile())
loan['Gender'] = loan['Gender'].fillna(1)
loan['Self_Employed'] = loan['Self_Employed'].fillna(0)
loan['Married'] = loan['Married'].fillna(0)
loan['Dependents'] = loan['Dependents'].fillna(loan['Dependents'].quantile())

loan_test['Credit_History'] = loan_test['Credit_History'].fillna(loan_test['Credit_History'].mean())
#loan_test['Credit_History'] = loan_test['Credit_History'].fillna(loan_test['Credit_History'].quantile())
loan_test['ApplicantIncome'] = loan_test['ApplicantIncome'].fillna(loan_test['ApplicantIncome'].quantile())
loan_test['LoanAmount'] = loan_test['LoanAmount'].fillna(loan_test['LoanAmount'].quantile())
loan_test['Loan_Amount_Term'] = loan_test['Loan_Amount_Term'].fillna(loan_test['Loan_Amount_Term'].quantile())
loan_test['Gender'] = loan_test['Gender'].fillna(1)
loan_test['Self_Employed'] = loan_test['Self_Employed'].fillna(0)
loan['Married'] = loan['Married'].fillna(0)
loan_test['Dependents'] = loan_test['Dependents'].fillna(0)

In [7]:
#print(loan['Married'].unique())
#print(loan['Property_Area'].unique())
#print(loan['Dependents'].unique())
#predictors = ['Married', 'Property_Area', 'Dependents', 'Self_Employed', 'Gender', 
#              'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term',
#              'Education']
#predictors = ['ApplicantIncome', 'CoapplicantIncome', 'Education', 'Married']
#selector = SelectKBest(f_classif, k=3)
#selector.fit(loan[predictors], loan['Loan_Status'])
#scores = -np.log10(selector.pvalues_)

# Plot the scores
#plt.bar(range(len(predictors)), scores)
#plt.xticks(range(len(predictors)), predictors, rotation='vertical')
#plt.show()

#selector = SelectKBest(chi2, k=3)
#selector.fit(loan[predictors], loan['Loan_Status'])
#scores = -np.log10(selector.pvalues_)

#plt.bar(range(len(predictors)), scores)
#plt.xticks(range(len(predictors)), predictors, rotation='vertical')
#plt.show()

In [8]:
loan_credit_history = loan_credit_history.dropna(subset=['Credit_History'])
print(loan_credit_history['Credit_History'].unique())
predictors = ['Education']#, 'CoapplicantIncome', 'Property_Area'] #'LoanAmount',
model = svm.SVC(probability=True)
scores = cross_validation.cross_val_score(model, loan_credit_history[predictors], 
                                          loan_credit_history['Credit_History'], cv=12)
print(scores)
print(scores.mean())
model.fit(loan_credit_history[predictors], loan_credit_history['Credit_History'])
loan['Credit_History_New'] = model.predict(loan[predictors])
loan_test['Credit_History_New'] = model.predict(loan_test[predictors])

[ 1.  0.]
[ 0.83333333  0.83333333  0.83333333  0.83333333  0.83333333  0.85106383
  0.85106383  0.84782609  0.84782609  0.84782609  0.84782609  0.84782609]
0.842327063419


In [9]:
predictors = ['Credit_History', 'Education', 'Married', 'CoapplicantIncome', 'Property_Area', 'LoanAmount']
classes = ['Y', 'N']
classifier = BernoulliNB(class_prior=[0, 1])  # Here we provide prior probabilities.
classifier.fit(data, classes)
model = BernoulliNB(class_prior=[])
scores = cross_validation.cross_val_score(model, loan[predictors], loan['Loan_Status'], cv=12)
print(scores)
print(scores.mean())

predictors = ['Credit_History', 'Education', 'Married']#, 'CoapplicantIncome', 'Property_Area'] #'LoanAmount',
model = svm.SVC(probability=True, shrinking=False)
scores = cross_validation.cross_val_score(model, loan[predictors], loan['Loan_Status'], cv=12)
print(scores)
print(scores.mean())
model.fit(loan[predictors], loan['Loan_Status'])

[ 0.80769231  0.84615385  0.74509804  0.80392157  0.76470588  0.78431373
  0.78431373  0.84313725  0.84313725  0.82352941  0.84313725  0.82352941]
0.809389140271
[ 0.80769231  0.84615385  0.74509804  0.80392157  0.76470588  0.78431373
  0.78431373  0.84313725  0.84313725  0.82352941  0.84313725  0.82352941]
0.809389140271


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=False,
  tol=0.001, verbose=False)

In [10]:
loan_test['Loan_Status'] = model.predict(loan_test[predictors])

In [11]:
loan_test_result = loan_test[['Loan_ID', 'Loan_Status']].copy()

In [12]:
loan_test_result.to_csv('result.csv', index=False)

In [15]:
plt.figure()
#loan['Loan_Amount_Term'].plot.hist()
#plt.show()
print(loan['Loan_Status'].value_counts())
loan['Loan_Status'].value_counts().plot(kind='bar')
plt.show()
#loan['Self_Employed'].value_counts().plot(kind='bar')
#plt.show()

Y    422
N    192
Name: Loan_Status, dtype: int64
